In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

import time as time

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import L2
import h5py

In [2]:
colab_flag = False

FTYPE = np.float32
ITYPE = np.int32

array = np.array
float32 = np.float32
int32 = np.int32
float64 = np.float64
int64 = np.int64

In [3]:
if colab_flag == True:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Github/MLROM/KS/')

In [4]:
print(os.getcwd())

/home/rkaushik/Documents/Thesis/MLROM/KS


In [5]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

if colab_flag == False:
    if gpus:
        gpu_to_use = 1
        tf.config.set_visible_devices(gpus[gpu_to_use], 'GPU')
        logical_devices = tf.config.list_logical_devices('GPU')
        print(logical_devices)

2022-11-30 23:53:43.180160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 23:53:43.180671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 23:53:43.294128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 23:53:43.294482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 23:53:43.294865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2022-11-30 23:53:43.297387: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 23:53:43.297891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 23:53:43.298537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 23:53:43.298935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [6]:
from tools.misc_tools import create_data_for_RNN
from tools.ae_v5 import Autoencoder
from tools.ESN_v1 import ESN
from tools.ESN_v3 import ESN as ESNv3

In [7]:
# print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())
print(tf.config.list_logical_devices())
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
[LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]
2.8.1


# KS System

In [8]:
gs_idx = '000'
dir_name_gs_all = os.getcwd() + '/grid_search/gridsearch_' + gs_idx

worker_id = 1

# worker_dir_list = os.listdir(dir_name_gs_all)
# worker_dir_list = [dir_name_gs_all+'/'+dname for dname in worker_dir_list if os.path.isdir(dir_name_gs_all+'/'+dname)]

worker_dir_list = [dir_name_gs_all+'/worker_{}'.format(worker_id)]

ESN_dir_list = []
for dname in worker_dir_list:
    temp = os.listdir(dname)
    temp = [dname+'/'+esndir for esndir in temp if os.path.isdir(dname+'/'+esndir)]
    ESN_dir_list.extend(temp)

In [9]:
# setting up params (and saving, if applicable)

# RNN directory
dir_name_rnn = ESN_dir_list[0]

# reading AE directory
with open(dir_name_rnn + '/sim_data_AE_params.txt') as f:
    lines = f.readlines()

params_dict = eval(''.join(lines))

dir_name_ae = params_dict['dir_name_ae']
ae_idx = dir_name_ae[-3:]
dir_name_ae = os.getcwd()+'/saved_ae/ae_'+ae_idx
try:
    use_ae_data = params_dict['use_ae_data']
except:
    print("'use_ae_data' not present in sim_data_AE_params, set to True.")
    use_ae_data = True

# reading RNN paramaters
with open(dir_name_rnn + '/RNN_specific_data.txt') as f:
    lines = f.readlines()

params_rnn_dict = eval(''.join(lines))



dt_rnn = params_rnn_dict['dt_rnn']
T_sample_input = params_rnn_dict['T_sample_input']
T_sample_output = params_rnn_dict['T_sample_output']
T_offset = params_rnn_dict['T_offset']
return_params_arr = params_rnn_dict['return_params_arr']
params = params_rnn_dict['params']
delta_t = params_rnn_dict['delta_t']
normalize_dataset = params_rnn_dict['normalize_dataset']
num_input_tsteps = params_rnn_dict['num_input_tsteps']
skip_intermediate = params_rnn_dict['skip_intermediate']
normalization_type = params_rnn_dict['normalization_type']

normalization_arr_rnn = None
if normalize_dataset == True:
    temp = np.load(dir_name_rnn+'/normalization_data.npz', allow_pickle=True)
    normalization_arr_rnn = temp['normalization_arr'][0]
    
# reading simulation parameters
with open(dir_name_ae + '/ae_data.txt') as f:
    lines = f.readlines()
params_dict = eval(''.join(lines))
data_dir_idx = params_dict['data_dir_idx']
normalizeforae_flag = params_dict['normalizeforae_flag']

temp = np.load(dir_name_ae+'/normalization_data.npz', allow_pickle=True)
normalization_constant_arr_aedata = temp['normalization_constant_arr_aedata'][0]

try:
    ae_data_with_params = params_dict['ae_data_with_params']
except:
    print("'ae_data_with_params' not present in ae_data, set to 'True'.")
    ae_data_with_params = True

print('dir_name_rnn:', dir_name_rnn)
print('dir_name_ae:', dir_name_ae)
print('data_dir_idx:', data_dir_idx)

# loading data
dir_name_data = os.getcwd() + '/saved_data/data_' + data_dir_idx
    
with open(dir_name_data + '/sim_data_params.txt') as f:
    lines = f.readlines()
params_dict = eval(''.join(lines))
params_mat = params_dict['params_mat']
# init_state_mat = params_dict['init_state_mat']
t0 = params_dict['t0']
T = params_dict['T']
delta_t = params_dict['delta_t']
numpoints_xgrid = params_dict['numpoints_xgrid']
length = params_dict['length']
return_params_arr = params_dict['return_params_arr']
normalize_flag_ogdata = params_dict['normalize_flag']
print('normalize_flag_ogdata:', normalize_flag_ogdata)
alldata_withparams_flag = params_dict['alldata_withparams_flag']

xgrid = length*np.linspace(0, 1, numpoints_xgrid)

fl = np.load(dir_name_data+'/data.npz', allow_pickle=True)
# all_data = fl['all_data']
# boundary_idx_arr = fl['boundary_idx_arr']
# normalization_constant_arr_ogdata = fl['normalization_constant_arr'][0]
# initial_t0 = fl['initial_t0']
# init_state_mat = fl['init_state_mat']
lyapunov_spectrum_mat = fl['lyapunov_spectrum_mat']

# training params
with open(dir_name_rnn + '/training_specific_params.txt') as f:
    lines = f.readlines()

tparams_dict = eval(''.join(lines))

# learning_rate_dict = tparams_dict['learning_rate_dict']
epochs = tparams_dict['epochs']
patience = tparams_dict['patience']
min_delta = tparams_dict['min_delta']
prng_seed = tparams_dict['prng_seed']
train_split = tparams_dict['train_split']
val_split = tparams_dict['val_split']
batch_size = tparams_dict['batch_size']

test_split = 1 - train_split - val_split

# setting seed for PRNGs
np.random.seed(prng_seed)
tf.random.set_seed(prng_seed)

dir_name_rnn: /home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_000/worker_1/ESN_030
dir_name_ae: /home/rkaushik/Documents/Thesis/MLROM/KS/saved_ae/ae_046
data_dir_idx: 005
normalize_flag_ogdata: False


In [10]:
lyapunov_time_arr = np.empty(shape=lyapunov_spectrum_mat.shape[0], dtype=FTYPE)
for i in range(lyapunov_spectrum_mat.shape[0]):
    lyapunov_time_arr[i] = 1/lyapunov_spectrum_mat[i, 0]
    print('Case : {}, lyapunov exponent : {}, lyapunov time : {}s'.format(i+1, lyapunov_spectrum_mat[i, 0], lyapunov_time_arr[i]))

Case : 1, lyapunov exponent : 0.06465670311438651, lyapunov time : 15.466300964355469s


In [11]:
def plot(
        boundary_idx_arr,
        delta_t,
        all_data,
        xgrid,
        xticks_snapto=20,
        num_yticks=11,
    ):

    n = len(boundary_idx_arr)
    # '''
    num_cols = 1
    num_rows = 1
    factor = 1
    # fig = plt.figure(figsize=(7.5*num_cols, 7.5*num_rows))

    num_modes = xgrid.shape[0]

    prev_idx = 0
    for i in range(len(boundary_idx_arr)):
        next_idx = boundary_idx_arr[i]
        fig, ax = plt.subplots(figsize=(factor*7.5*num_cols, factor*5.0*num_rows))
        N = next_idx - prev_idx
        input_time = np.arange(0, N)*delta_t

        im = ax.imshow(all_data[prev_idx:next_idx, 0:num_modes].transpose(), aspect='auto', origin='lower')
        num_xticks = 1 + int((N*delta_t + 0.5*xticks_snapto) // xticks_snapto)
        # xticks = np.linspace(0, N, num_xticks, dtype=np.int32)
        xticks = np.arange(0, N, int((xticks_snapto+0.5*delta_t)//delta_t))
        ax.set_xticks(ticks=xticks)
        ax.set_xticklabels(np.round(xticks*delta_t, 1))
        ax.tick_params(axis='x', rotation=270+45)

        yticks = np.linspace(0, 1, num_yticks)*(len(xgrid)-1)
        yticklabels = np.round(np.linspace(0, 1, yticks.shape[0])*xgrid[-1], 2)
        ax.set_yticks(ticks=yticks)
        ax.set_yticklabels(yticklabels)

        ax.set_xlabel('Time')
        ax.set_ylabel(r'$x$')
        # ax.title.set_text(r'Latent States')

        plt.colorbar(im)
        plt.show()
        print('')

        prev_idx = next_idx

    # '''

In [12]:
# # dealing with normalizing the data before feeding into autoencoder
# if normalizeforae_flag == True and use_ae_data == True:
#     for i in range(numpoints_xgrid):
#         all_data[:, i] -= normalization_constant_arr_aedata[0, i]
#         all_data[:, i] /= normalization_constant_arr_aedata[1, i]
# if ae_data_with_params == False:
#     all_data = all_data[:, 0:numpoints_xgrid]

# Autoencoder

In [13]:
# load_file = dir_name_ae+dir_sep+'new_class_data'+dir_sep+'ae_'+ae_idx+'_class_dict.txt'
# wt_file = dir_name_ae+dir_sep+'new_class_data'+dir_sep+'ae_'+ae_idx+'_ae_weights.h5'
# if use_ae_data == True:
#     load_file = dir_name_ae+'/final_net/final_net_class_dict.txt'
#     wt_file = dir_name_ae+'/final_net/final_net_ae_weights.h5'

In [14]:
# if use_ae_data == True:
#     ae_net = Autoencoder(all_data.shape[1], load_file=load_file)
#     ae_net.load_weights_from_file(wt_file)

In [15]:
# # create data
# if use_ae_data == True:
#     latent_states_all = ae_net.encoder_net.predict(all_data)
#     del(all_data)
# else:
#     latent_states_all = all_data[:, 0:xgrid.shape[0]]

# num_latent_states = latent_states_all.shape[1]

In [16]:
# plot_latent_states_KS(
#     boundary_idx_arr,
#     latent_states_all,
#     delta_t,
#     dir_name_ae,
#     xticks_snapto=int(40*np.round((T//10)/40)),
#     num_yticks=11,
#     save_figs=False)

# ESN

In [17]:
for dname_idx in range(len(ESN_dir_list)):
    computation_time = time.time()
    dir_name_rnn = ESN_dir_list[dname_idx]

    sidx1 = dir_name_rnn[::-1].index('/')
    sidx2 = dir_name_rnn[-sidx1-2::-1].index('/')
    print(dir_name_rnn[-(sidx1+sidx2+1):])

    # Initialize ESN_v1 network
    load_file = dir_name_rnn + '/final_net/final_net_class_dict.txt'
    rnn_net = ESN(
        load_file=load_file,
        stddev=0.0,
    )
    rnn_net.build(input_shape=(1, 1, rnn_net.data_dim))

    wt_file = dir_name_rnn+'/final_net/final_net_ESN_weights.h5'
    rnn_net.load_weights_from_file(wt_file)

    new_ESN = ESNv3(load_file=load_file)
    new_ESN.build(input_shape=(1, 1, rnn_net.data_dim))

    for i in range(len(rnn_net.ESN_layers)):
        rnn_net_cell = rnn_net.ESN_layers[i].cell
        input_size = rnn_net_cell.input_size
        usebias_Win = rnn_net.usebias_Win[i]

        new_ESN_cell = new_ESN.ESN_layers[i].cell

        # setting Win
        K.set_value(new_ESN_cell.Win.kernel, rnn_net_cell.Win_np[0:input_size, :])
        if usebias_Win == True:
            K.set_value(new_ESN_cell.Win.bias, rnn_net_cell.Win_np[-1, :])

        # setting Wres
        K.set_value(new_ESN_cell.Wres.kernel, rnn_net_cell.Wres_np)

    # setting Wout
    usebias_Wout = rnn_net.usebias_Wout
    Wout_np = rnn_net.Wout.numpy()

    K.set_value(new_ESN.Wout.kernel, Wout_np[0:rnn_net.ESN_layers_units[-1], :])
    if usebias_Wout == True:
        K.set_value(new_ESN.Wout.bias, Wout_np[-1, :])

    new_ESN.save_model_weights(dir_name_rnn+'/final_net/final_net_ESN_weights')
    
    print('    time taken : {} s'.format(time.time()-computation_time))

worker_1/ESN_030
    time taken : 154.82240915298462 s
worker_1/ESN_002
    time taken : 160.83131980895996 s
worker_1/ESN_076
    time taken : 179.15672206878662 s
worker_1/ESN_044
    time taken : 171.168874502182 s
worker_1/ESN_029
    time taken : 20.873427152633667 s
worker_1/ESN_041
    time taken : 19.87019443511963 s
worker_1/ESN_007
    time taken : 14.942042112350464 s
worker_1/ESN_023
    time taken : 12.445675373077393 s
worker_1/ESN_045
    time taken : 11.770596027374268 s
worker_1/ESN_047
    time taken : 22.14444375038147 s
worker_1/ESN_055
    time taken : 20.50893807411194 s
worker_1/ESN_071
    time taken : 13.835004091262817 s
worker_1/ESN_054
    time taken : 163.01694798469543 s
worker_1/ESN_057
    time taken : 18.637277841567993 s
worker_1/ESN_042
    time taken : 162.94983053207397 s
worker_1/ESN_038
    time taken : 164.66737961769104 s
worker_1/ESN_075
    time taken : 14.664850950241089 s
worker_1/ESN_034
    time taken : 164.25978827476501 s
worker_1/ESN_05

In [18]:
# new_ESN(np.ones(shape=(1, 1, rnn_net.data_dim)), training=False)